# Introdução

### Objetivo

Esse notebook terá como meta analisar quais fatores contribuem para que um funcionário de uma empresa seja demitido (ou se demita), além de criar um modelo que possa prever as  demissões.

Toda essa análise visará a sugestão de um **plano de retenção de funcionários**, buscando evitar a saida dos mesmos.

# Bibliotecas necessárias

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as matplot
import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats as stats
import statsmodels.api as sm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import accuracy_score, classification_report, precision_score, recall_score, confusion_matrix, precision_recall_curve
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import RobustScaler

from sklearn.tree import DecisionTreeClassifier
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier

# Carregamento dos dados

In [ ]:
df = pd.read_csv('../input/hr-analytics/HR_comma_sep.csv')
df.head()

Entenda as colunas como:

- **satisfaction_level**: nível de satisfação do funcionário (o quanto ele está feliz com seu emprego)
- **last_evaluation**: última avaliação que o funcionário recebeu (uma nota entre 0 e 1, onde 1 representa os funcionários bem avaliados)
- **number_project**: números de projetos que o funcionário trabalha
- **average_montly_hours**: média de horas trabalhadas mensalmente por esse funcionário
- **time_spend_company**: tempo que o funcionário está na empresa (medido em anos)
- **Work_accident**: se o funcionário já sofreu algum acidente de trabalho ou não
- **left**: se o funcionário continua trabalhando (0) ou foi demitido (1)
- **promotion_last_5years**: se o funcionário recebeu alguma promoção (elevação de cargo) nos últimos 5 anos
- **Department**: departamento em que o funcionário trabalha
- **salary**: salário do funcionário (note que, ao invés de números, temos **alto, médio** ou **baixo**)

É importante lembrar que a coluna **left** nos informa que o funcionário saiu da empresa, porém não especifica se ele foi demitido ou se demitiu.

# Análise dos dados

In [ ]:
# Valores nulos
df.isnull().sum()

Como esperado de um conjunto de dados do Kaggle, não há valores nulos

In [ ]:
# Renomear alguns colunas para melhor leitura
df = df.rename(columns={
    'Work_accident': 'work_accident',
    'promotion_last_5years': 'promotion_last_5_years',
    'Department': 'department'
})

In [ ]:
# Mover a coluna "left" (classe) para a primeira coluna da tabela
front = df['left']
df.drop(labels=['left'], axis=1, inplace=True)
df.insert(0, 'left', front)
df.head()

### Exploração profunda

In [ ]:
# Dimensões
df.shape

In [ ]:
# Tipos
df.dtypes

In [ ]:
# Percentual de demissões
left_rate = df.left.value_counts() / len(df)
left_rate

In [ ]:
# Alguns dados sobre as demissões
left_summary = df.groupby('left')
left_summary.mean()

Observando os dados acima, podemos notar que:
- A empresa possui certa de 15.000 funcionários
- O percentual de demissões é de aproximadamente 24 %
- O nível de satisfação médio de seus funcionários é de aproximadamente 67 % 

In [ ]:
# Descrição estatística dos dados
df.describe().T

### Correlações

In [ ]:
# Matriz de correlação
corr = df.corr()
corr = (corr)

plt.figure(figsize=(8, 6))

sns.heatmap(corr, vmin=-1, vmax=1, annot=True, cmap='coolwarm_r')

plt.show()

Nenhum das "features" possui correlação positiva forte (igual ou acima de 0.5) ou negativa forte (igual ou abaixo de -0.5), porém podemos observar algumas "features" com correlações moderadas:
- Positiva: **last_evaluation** e **number_project**, **average_montly_hours** e **number_project**, **last_evaluation** e **last_evaluation**
- Negativa: **satisfaction_level** e **left**

Se pararmos para pensar nas correlações positivas, elas nos dizem que **quando mais um funcionário trabalha mensalmente, mas projetos eles faz (e vice-versa) e melhor a sua avaliação.**

E sobre a correlação negativa, faz todo o sentido que, **quanto menor o nível de satisfação de um funcionário, maior a chance dele pedir demissão ou ser demitido.**

### Teste estatístico

**T-Test com uma amostra (mensura do nível de satisfação)**: esse teste verifica se a média da amostra difere da média da população. Como **satisfaction_level** possui a maior correlação com a classe **left**, vamos tesr se a média com funcionários que foram demitidos difere da média dos que não foram demitidos.

Teste de hipótese:
- **Hipótese nula**: NÃO HÁ DIFERENÇA no nível de satisfação (satisfaction_level) entre funcionários que foram demitidos e os que não foram demitidos
- **Hipótese alternativa**: HÁ DIFERENÇA no nível de satisfação (satisfaction_level) entre funcionários que foram demitidos e os que não foram demitidos

In [ ]:
# Populaçao dos funcionários
emp_population = df['satisfaction_level'][df['left'] == 0].mean()

# População dos funcionários demitidos
emp_left_population = df['satisfaction_level'][df['left'] == 1].mean()

print('Satisfação média dos funcionários que NÃO FORAM demitidos:',emp_population)
print('Satisfação média dos funcionários que FORAM demitidos: ', emp_left_population)

#### Conduzindo o T-Test

Iremos fazer um T-Test com um **nível de confiança de 95%** e verificar se devemos rejeitar a hipótese nula de que a amostra possui a mesma distribuição da população de funcionários.

In [ ]:
stats.ttest_1samp(a=df[df['left'] == 1]['satisfaction_level'], popmean=emp_population)

O resultado do teste nos mostra que a estatística do T-Test é aproximadamente **-51.33**. Esse número representa o quando a média da amostra difere da hipótese nula. Se a **estatística t** estiver fora dos quartis da distribuição e do grau de confiança correspondente ao nosso nível de confiança, rejeitamos a hipótese nula.

In [ ]:
# Grau de confiança
degree_freedom = len(df[df['left'] == 1])
# Quartil esquerdo
LQ = stats.t.ppf(0.025, degree_freedom)
# Quartil direito
RQ = stats.t.ppf(0.975, degree_freedom)

print('Distrubuição t LQ: ', LQ)
print('Distrubuição t RQ: ', RQ)

#### Conclusão do T-Test

No fim das contas, podemos **rejeitar a hipótese nula**, devido à:
- O score do T-Test está fora dos quartis
- O P-value é menor que o nível de confiança (5%)

Baseado na análise estatística de uma amostra, podemos ver que há diferença significante entre a média de satisfação dos funcionários que se demitiram e da população de funcionários. O valor extremamente baixo do P-value (0.0) é um grande indicator da rejeição da hipótese nula.

Porém, isso pode não ter uma significância prática, e precisamos conduzir mais experimentos ou coletar mais dados visando conclusões mais acuradas.

### Visualização das distribuições

In [ ]:
f, ax = plt.subplots(ncols=3, figsize=(15, 5))

# satisfaction_level
sns.distplot(df.satisfaction_level, color='g', ax=ax[0]).set_title('Distribuição do nível de satisfação')
ax[0].set_ylabel('Quantidade')

# last_evaluation
sns.distplot(df.last_evaluation, color='r', ax=ax[1]).set_title('Distribuição da avaliação')

# average_montly_hours
sns.distplot(df.average_montly_hours, color='b', ax=ax[2]).set_title('Distribuição da média de horas trabalhadas mensalmente')

plt.tight_layout()
plt.show()

Podemos concluir que:
- **Nível de satisfação**: há um grande pico entre funcionários com satisfação baixa e satisfação alta
- **Avaliação**: há uma distribuição bimodal entre funcionários com baixa avaliação (0.6) e alta avaliação (0.8)
- **Média de horas trabalhadas mensalmente**: outra distribuição bimodal,  entre funcionários com quantidade baixa e alta da média de horas trabalhadas mensalmente (abaixo de 150 horas e acima de 250 horas)
- Os gráficos 2 e 3 possuem distribuições similares
- Como dito anteriormente, funcionários com médias menores de horas trabalhadas possuem uma avaliação menor e vice-versa.

### Salário vs Demissão

In [ ]:
plt.figure(figsize=(10, 4))

sns.countplot(y='salary', hue='left', palette=['b', 'r'], data=df)
plt.title('Salário vs Demissão')
plt.grid(axis='x')

plt.show()

Grande parte dos funcionários foram demitidos possuiam salários **baixos** e **médios**, enquanto aqueles com salários **altos** quase não foram demitidos. Funcionários com **baixos salários** são os que mais tendem a sair da empresa.

### Departamento vs Demissão

In [ ]:
plt.figure(figsize=(12, 6))

sns.countplot(x='department', data=df)
plt.title('Departamentos')
plt.xlabel('Departamentos')
plt.xticks(rotation=-45)
plt.grid(axis='y')

plt.show()

In [ ]:
plt.figure(figsize=(12, 5))

sns.countplot(y='department', hue='left', palette=['b', 'r'], data=df)
plt.title('Demissões por departamento')

plt.show()

Os departamentos **sales**, **technical** e **support** possuem as maiores quantidades de  funcionários que pedem demissão, enquanto o departamento **management** possui a menor quantidade.

### Demissão vs Número de projetos

In [ ]:
plt.figure(figsize=(10, 6))

sns.barplot(x='number_project', y='number_project', hue='left', palette=['blue', 'red'], data=df, estimator=lambda x: len(x) / len(df) * 100)
plt.ylabel('Percentual')

plt.show()

É possível observar que funcionários com **2, 6 e 7 projetos** foram os que mais sairam da empresa. A grande maioria dos funcionários com **3, 4 e 6 projetos** não sairam. É importante notar que todos os funcionários com **7 projetos** sairam da empresa, e há uma tendência crescendo nas demissões à medida que a quantidade de projetos aumenta (observando a partir de 3 projetos).

O fato de que uma grande quantidade de funcionários com apenas 2 projetos sairem da companua pode ter sido ocasionado pelo fato de que, devido a essa quantidade baixa de projetos, eles não receberam a devida **avaliação** (last_evaluation). E os funcionários com muitos projetos (**5 a 7**) podem ter ficado sobrecarregados.

### Demissão vs Avaliação

In [ ]:
plt.figure(figsize=(15, 4))

sns.kdeplot(df.loc[(df['left'] == 0), 'last_evaluation'], shade=True, label='Não demitido')
sns.kdeplot(df.loc[(df['left'] == 1), 'last_evaluation'], color='r', shade=True, label='Demitido')
plt.title('Distribuição dos funcionários \nNão demitido vs Demitido')
plt.show()

Há uma distribuição bimodal entre os funcionários demitidos, em que funcionários com performance **baixa** tendem a deixar a empresa, e aqueles com performance **alta** também tendem a deixar a empresa. Aqueles entre 0.6 a 0.8 tendem a continuar.

### Demissão vs Média de horas mensais

In [ ]:
plt.figure(figsize=(12, 5))

sns.kdeplot(df.loc[(df['left'] == 0), 'average_montly_hours'], color='b', shade=True, label='Não demitido')
sns.kdeplot(df.loc[(df['left'] == 1), 'average_montly_hours'], color='r', shade=True, label='Demitido')
plt.xlabel('Média de horas mensais')
plt.title('Distribuição de horas mensais trabalhadas \nNão demitido vs Demitido')

plt.show()

Há uma distribuição bimodal entre funcionários demitidos, onde funcionários com uma média mensal de horas trabalhadas **menor que 150 horas** ou **maiores que 250 horas** possuem uma tendência maior em sair da empresa.

### Demissão vs Satisfação

In [ ]:
plt.figure(figsize=(12, 5))

sns.kdeplot(df.loc[(df['left'] == 0), 'satisfaction_level'], color='b', shade=True, label='Não demitido')
sns.kdeplot(df.loc[(df['left'] == 1), 'satisfaction_level'], color='r', shade=True, label='Demitido')
plt.xlabel('Satisfação')
plt.title('Distribuição da satisfação \nNão demitido vs Demitido')

plt.show()

Há uma distribuição trimodal entre funcionários demitidos. Funcionários com **satisfação entre 0.2 a 0.5** e com **satisfação maior que 0.7** possuem uma tendência maior em sair da empresa.

### Quantidade de projetos vs Média de horas mensais 

In [ ]:
plt.figure(figsize=(10, 5))

sns.boxplot(x='number_project', y='average_montly_hours', hue='left', palette=['lightblue','r'], data=df)

plt.show()

É possível notar que os funcionários NÃO DEMITIDOS possuem uma média de horas mensais trabalhadas quase constante, mesmo que o número de projetos aumente. Já os funcionários DEMITIDOS tendem a ter uma média de horas mensais trabalhadas que cresce de acordo com o número de projetos.

### Quantidade de projetos vs Avaliação

In [ ]:
plt.figure(figsize=(10, 5))

sns.boxplot(x='number_project', y='last_evaluation', hue='left', palette=['lightblue','r'], data=df)

plt.show()

Aqui temos um resultado meio estranho: funcionários com mais projetos e que FORAM DEMITIDOS tendem a possuir uma avaliação maior, enquanto os funcionários que NÃO FORAM DEMITIDOS possuem uma avaliação quase constante, mesmo com um número de projetos maior. Surjem duas dúvidas:
- Funcionários com avaliações menores não deveriam FICAR na empresa?
- Os funcionários com avaliações menores não deveriam SAIR da empresa?

### Satisfação vs Avaliação

In [ ]:
plt.figure(figsize=(10, 6))

sns.lmplot(x='satisfaction_level', y='last_evaluation', palette=['lightblue','r'], data=df, fit_reg=False, hue='left')

plt.show()

É possível observar **3 grupos distintos**:

- **Grupo 1**: Funcionários com avaliações maiores que 0.75, porém com um nível de satisfação menor que 0.2. Isso pode indicar funcionários que trabalham muito, talvez até sejam sobrecarregados, e são bem avaliados, porém estão insatisfeitos com algum motivos, como a sobrecarga de trabalho citada anteriormente.
- **Grupo 2**: Funcionários com avaliações menores que 0.58 e nível de satisfação entre 0.35 e 0.45, ou seja, mal avaliados e insatisfeitos.
- **Grupo 3**: Funcionários com avaliações maiores que 0.8 e nivel de satisfação entre 0.7 e 1.0. Esse é o tipo ideal de funcionário, que trabalha muito e feliz. Talvez eles tenham saido por causa de novas oportunidades de emprego.

### Demissão vs Anos na empresa

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='time_spend_company', y='time_spend_company', hue='left', palette=['b', 'r'], data=df, estimator=lambda x: len(x) / len(df) * 100)
plt.ylabel('Porcentagem')

plt.show()

Grande parte dos funcionários sai da empresa entre **3 e 5** anos de trabalho, sendo que mais da metade dos funcionários com **4 e 5** anos de trabalho sairam.

### K-Means das demissões

In [ ]:
# Graph and create 3 clusters of Employee Turnover
kmeans = KMeans(n_clusters=3,random_state=2)

kmeans.fit(df[df.left==1][["satisfaction_level","last_evaluation"]])

kmeans_colors = ['green' if c == 0 else 'blue' if c == 2 else 'red' for c in kmeans.labels_]

fig = plt.figure(figsize=(10, 6))

plt.scatter(x="satisfaction_level",y="last_evaluation", data=df[df.left==1], alpha=0.25,color = kmeans_colors)
plt.xlabel("Nível de satisfação")
plt.ylabel("Avaliação")
plt.scatter(x=kmeans.cluster_centers_[:,0],y=kmeans.cluster_centers_[:,1],color="black",marker="X",s=100)
plt.title("Clusters dos funcionários demitidos")

plt.show()

- Cluster 1 (Azul): funcionários bem avaliados, porém tristes
- Cluster 2 (Verde): funcionários mal avaliados e tristes
- Cluster 3 (Vermelho): funcionários bem avaliados e felizes

É importante notar que estamos pressupondo que há 3 clusters, mas isso não quer dizer que seja a quantidade correta.

# Importância das "features"

In [ ]:
plt.rcParams['figure.figsize'] = (12,6)

df['department'] = df['department'].astype('category').cat.codes
df['salary'] = df['salary'].astype('category').cat.codes

X=df.drop('left', 1)
y=df['left']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

tree = DecisionTreeClassifier(class_weight='balanced', min_weight_fraction_leaf=0.01)

model = tree.fit(X_train, y_train)

importances = model.feature_importances_
feat_names = df.drop('left', 1).columns

indices = np.argsort(importances)[::-1]

plt.title('Importância das "features"')
plt.bar(range(len(indices)), importances[indices], color='blue', align='center')
plt.step(range(len(indices)), np.cumsum(importances[indices]), where='mid', label='Cumulativo')
plt.xticks(range(len(indices)), feat_names[indices], rotation='vertical', fontsize=14)
plt.xlim([-1, len(indices)])
plt.grid(True)

plt.show()

Utilizando o modelo "Decision tree classifier", podemos ranquear as "features" com maior importância durante a predição, que nesse caso, o top 3 consiste em:
1. satisfaction_level
2. time_spend_company
3. last_evaluation

# Modelagem

In [ ]:
df['int'] = 1
indep_var = ['satisfaction_level', 'last_evaluation', 'time_spend_company', 'int', 'left']
df = df[indep_var]

In [ ]:
X = df.drop('left', 1)
y = df.left

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
iv = ['satisfaction_level', 'last_evaluation', 'time_spend_company', 'int']

log_reg = sm.Logit(y_train, X_train[iv])
answer = log_reg.fit()

answer.summary()
answer.params

In [ ]:
coef = answer.params

def y(coef, satisfaction_level, last_evaluation, time_spend_company):
    return coef[3] + coef[0] * satisfaction_level + coef[1] * last_evaluation + coef[2] * time_spend_company

y1 = y(coef, 0.7, 0.8, 3)
p = np.exp(y1) / (1 + np.exp(y1))
p

### Modelo base

In [ ]:
def base_model(X):
    y = np.zeros(X.shape[0])
    
    return y

In [ ]:
X = df.drop('left', 1)
y = df.left

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.15, random_state=42, stratify=y)

In [ ]:
y_base = base_model(X_test)
print('Acurácia base', round(accuracy_score(y_test, y_base), 3))

### Logistic Regression

In [ ]:
model = LogisticRegression(penalty='l2', C=1)

model.fit(X_train, y_train)

preds = model.predict(X_test)
print('Acurácia de Logistic Regression', round(accuracy_score(y_test, preds), 3))

### Validação cruzada

In [ ]:
kfold = KFold(n_splits=10, random_state=42)

cv_model = LogisticRegression(class_weight='balanced')
results = cross_val_score(cv_model, X_train, y_train, cv=kfold, scoring='roc_auc')

print('AUC: {:.3f} ({:.3f})'.format(results.mean(), results.std()))

### Teste com vários modelos

In [ ]:
print('----------- Base --------------')
base_roc_auc = roc_auc_score(y_test, base_model(X_test))
print('AUC base = {:.2f}'.format(base_roc_auc))
print(classification_report(y_test, base_model(X_test)))

logis = LogisticRegression(class_weight = "balanced")
logis.fit(X_train, y_train)
print ("\n\n ------ Logistic Regression ---------")
logit_roc_auc = roc_auc_score(y_test, logis.predict(X_test))
print ("Logistic AUC = %2.2f" % logit_roc_auc)
print(classification_report(y_test, logis.predict(X_test)))


# Decision Tree 
dtree = DecisionTreeClassifier(class_weight="balanced", min_weight_fraction_leaf=0.01)
dtree = dtree.fit(X_train,y_train)
print ("\n\n ------ Decision Tree ------")
dt_roc_auc = roc_auc_score(y_test, dtree.predict(X_test))
print ("Decision Tree AUC = %2.2f" % dt_roc_auc)
print(classification_report(y_test, dtree.predict(X_test)))


# Random Forest
rf = RandomForestClassifier(
    n_estimators=1000, 
    max_depth=None, 
    min_samples_split=10, 
    class_weight="balanced" 
    )
rf.fit(X_train, y_train)
print ("\n\n ------Random Forest ------")
rf_roc_auc = roc_auc_score(y_test, rf.predict(X_test))
print ("Random Forest AUC = %2.2f" % rf_roc_auc)
print(classification_report(y_test, rf.predict(X_test)))


# Ada Boost
ada = AdaBoostClassifier(n_estimators=400, learning_rate=0.1)
ada.fit(X_train,y_train)
print ("\n\n ------- AdaBoost ------")
ada_roc_auc = roc_auc_score(y_test, ada.predict(X_test))
print ("AdaBoost AUC = %2.2f" % ada_roc_auc)
print(classification_report(y_test, ada.predict(X_test)))

### Gráfico ROC

In [ ]:
plt.figure(figsize=(10, 6))

fpr, tpr, thresholds = roc_curve(y_test, logis.predict_proba(X_test)[:,1])
rf_fpr, rf_tpr, rf_thresholds = roc_curve(y_test, rf.predict_proba(X_test)[:,1])
dt_fpr, dt_tpr, dt_thresholds = roc_curve(y_test, dtree.predict_proba(X_test)[:,1])
ada_fpr, ada_tpr, ada_thresholds = roc_curve(y_test, ada.predict_proba(X_test)[:,1])

plt.figure()

# Plot Logistic Regression ROC
plt.plot(fpr, tpr, label='Logistic Regression (area = %0.2f)' % logit_roc_auc)

# Plot Random Forest ROC
plt.plot(rf_fpr, rf_tpr, label='Random Forest (area = %0.2f)' % rf_roc_auc)

# Plot Decision Tree ROC
plt.plot(dt_fpr, dt_tpr, label='Decision Tree (area = %0.2f)' % dt_roc_auc)

# Plot AdaBoost ROC
plt.plot(ada_fpr, ada_tpr, label='AdaBoost (area = %0.2f)' % ada_roc_auc)

# Plot Base Rate ROC
plt.plot([0,1], [0,1],label='Base Rate')

plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Graph')
plt.legend(loc="lower right")
plt.show()

### Interpretação dos dados

Os funcionários que possuem mais chance se sair da empresa (através de uma demissão por justa causa ou saída voluntária) são aqueles que:
1. Estão com **trabalho insuficiente** (menor que 150h/mês ou 6h/dia)
2. Estão com **trabalho demais** (maior que 250h/mês ou 10h/dia)
3. Possuem **avaliações muito baixas ou muito altas**
4. Recebem **salários baixos ou médios**
5. Trabalham em **2, 6 ou 7** projetos
6. Já estão na empresa há 4 ou 5 anos

Além disso, ficou comprovado que o **nível de satisfação** é o maior indicador quando se trata do funcionário sair da empresa, seguido de **anos na empresa** e **avaliação**.

### Soluções em potencial

#### Funcionários com baixas avaliações

Como visto na análise dos dados, é comum funcionários com avaliações mais baixas sairem da empresa, e isso provavelmente ocorre através de demissões por justa causa, porém é uma boa ideia verificar corretamente os motivos por trás dessas baixas avaliações, afinal vimos que pode ser que eles recebam poucos projetos, o que dificulta a demonstração dos seus potênciais, e também pode haver uma certa **sobrecarga de trabalho**, o que pode acabar gerando estresse, outro fator que pode prejudicar a qualidade dos projetos e afins. Mas por quê investir tempo em funcionários com baixas avaliações? Muitas vezes, o custo de de fazer um novo processo seletivo e explicar todo o funcionamento da empresa para novos funcionários é maior do que investir em um plano para melhorar as avaliações dos funcionários já existentes. Então esse assunto e cálculos desses custos devem ser discutidos com os responsáveis (setor de RH, por exemplo).

#### Funcionários com baixas avaliações

Vimos também que é comum funcionários com avaliações mais altas (e até mesmos satisfeitos com seus empregos) deixarem a empresa. Um motivo para isso ocorrer pode ser a **falta de valorização** por parte da empresa, afinal, dificilmente alguém gosta de se esforçar no trabalho e alcançar resultados cada vez melhor ser receber algum incentivo por isso. Esse cenário pode trazer diversos prejuízos à empresa, já que talentos estaram saindo e podem até acabar trabalhando para concorrente.

### Abordagem

Classificar e ranquear funcionários através das probabilidade de saída, como visto abaixo:

In [ ]:
preds = logis.predict_proba(X_test)
pd.DataFrame(preds, columns=['Not left', 'Left']).sort_values(by=['Left'], ascending=False).head()

No exemplo acima, utilizamos o modelo **Logistic Regression** e fazendo uma previsão com os dados de teste, podemos visualizar quais os 5 funcionários com maior probabilidade de deixar a ampresa.

Com os dados das probabilidades em mãos, o setor responsável (geralmente o RH) poderia traçar planos para abordar esses funcionários e garantir sua permanência, como por exemplo:
- Planos de carreira
- Incentivos financeiros
- Palestras motivacionais
- Dentre outros benefícios.

Porém, é interessante lembrar que podem existir casos de **funcionários com alta probabilidade de deixar a empresa, mas que acabam não saindo depois de um tempo** (falsos positivos), algo que pode gerar um custo desnecessário há empresa, além dos **funcionários com baixa probabilidade de deixarem a empresa, mas que acabam saindo depois de um tempo**(falsos negativos), que, como dito anteriormente, podem acarretar na perda de talentos e de custos maiores para a realização de novas contratações.

# Referências

Os créditos desse notebook pertencem à [**Randy Lao**](https://www.kaggle.com/randylaosat) e ao seu trabalho [Predicting Employee Kernelover](https://www.kaggle.com/randylaosat/predicting-employee-kernelover/notebook), que me serviu como base para estudo e construção da conclusão.